|<h2>Course:</h2>|<h1><b><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></b></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Identifing latent factors<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Laminar profile of autoencoder sparsity<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, GPT2Tokenizer

# Exercise 1: Import text, tokenize, get MLP activations

In [ ]:
# model & tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model     = AutoModelForCausalLM.from_pretrained('gpt2')

# push to GPU in eval mode


In [ ]:
# a function to hook the activations


def implant_hook(layer_number):
  def hook(module, input, output):

  return hook

# put hooks in all layers


In [ ]:
import requests
text = requests.get('https://en.wikipedia.org/wiki/Light-emitting_diode').text




In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# tokenize the text
tokens =
print(f'T

In [ ]:
u,c =
sidx = np.argsort
u = u
c = c[

print('Top 30 most common tokens:')
for i in range(30):
  print(

In [ ]:
# create 10 batches of 1024 tokens
batches =
batches.shape

In [ ]:
# Forward-pass the tokens to get the activations


# Exercise 2: Create an autoencoder class in a function

In [ ]:
def createTheSAE(num_latent):

  # create the class
  class SparseAE(nn.Module):
    def __init__(self, input_dim, latent_dim, k=None, sparsity_weight=1, decor_weight=.0005):
      super().__init__()
      self.encoder = nn.Linear( , , bias=False)
      # self.decoder = nn.Linear(latent_dim, input_dim, bias=False)
      # note: decoder is tied to encoder in forward()

      self.sparsity_weight =
      self.decor_weight =

      # k-sparse parameter defaults to 50% of input
      if k==None:
        self.k =
      else:
        self.k =

    def forward(self, x):

      # forward pass to the latent layer
      latent = F.relu(self.

      # "k-sparsify": force sparsity by zeroing out small activations
      topk_vals = torch.topk(
      thresh = topk_vals[]. # kth-largest value is the smallest of the sorted top-k
      mask =  # mask is 0's and 1's
      latent_sparse =  # multiply the latent activations by the mask

      # finally, decode via tied weights (hint: transpose)
      y = F.linear(, )

      return y,latent_sparse

    def sparsity_loss(self, z):
      return self.sparsity_weight * # L1 loss

    # penalty on inter-latent covariance
    def decorrelation_loss(self, estLatent):
      cov = torch.cov(estLatent.T)
      off_diag = # just the off-diagonal elements
      return self.decor_weight * # sum of squared off-diagonal covariances



  # create an instance of the autoencoder
  ae = SparseAE(input_dim=X.shape[1], k=num_latent//3, latent_dim=
  ae = ae.to(device)
  return ae

In [ ]:
### test with some data

# get an MLP layer activation
nhidden =
X = activations['mlp_3'] # need to reshape

# create an SAE model instance (using a 2x expansion for the latent layer)
aemodel = createTheSAE(
aemodel = aemodel.to(device)

# and push some data through
aemodel(X)

# Exercise 3: Train layer-specific SAEs

In [ ]:
# takes ~3 mins for gpt2-small on GPU
# takes ~20 mins for gpt2-large on GPU

# initialize
latentDensity = np.zeros
densityActivation =
finalloss =

n_epochs =


## loop over layers
for layeri in

  # get the activations from this layer, and reshape to (N x hidden)
  nhidden = # don't forget to skip the first token :)
  X = activations[f'mlp_{layeri}'].reshape

  # create an SAE model for this layer (using a 2x expansion for the latent layer)
  aemodel = createTheSAE(
  aemodel = aemodel.to(device)

  ### train the model
  optimizer = optim.Adam(
  mse_loss  =

  for epoch in

    # forward pass
    optimizer.zero_grad()
    x_pred,latent =

    # backprop
    loss = # calculate and sum the two losses
    loss.backward()
    optimizer.step()

  # final loss after all epochs
  finalloss[layeri] =



  ## final run to get latent activations
  with torch.no_grad():
    aeout,latent = aemodel(X)

  # convert to numpy and back to CPU
  latent =


  ### latent layer characteristics
  # mask for zero-valued activations
  densitymask = np.full(..,np.nan)
  densitymask[latent!=0] =

  # density is the percent of nonzero activations per latent component
  densityPerComponent =
  latentDensity[layeri] = # average the density scores

  # token-averaged activation magnitude, excluding zeros
  nonzeroAct =
  nonzeroAct[np.isnan(nonzeroAct)] =

  # average nonzero activations per latent component (after minmax scaling)
  dpc = # min-max scale
  nza = # min-max scale
  densityActivation[layeri] =

  print(f'Finished layer {layeri+1:2}/{model.config.n_layer}')

# Exercise 4: Visualize SAE laminar profiles

In [ ]:
# plot the results
_,axs = plt.subplots(1,3,figsize=(12,3.5))

axs[0].plot(,'ks',markerfacecolor=[.7,.7,.9],markersize=9)
axs[0].set(xlabel='Layer',ylabel='MSE Loss',title='Final SAE loss per layer')

axs[1].plot(
axs[1].set(xlabel='Layer',ylabel='Density (% nonzero activations)',title='Latent density per layer')

axs[2].plot(
axs[2].set(xlabel='Layer',ylabel='Density $\\times$ activation',title='(Density $\\times$ activation) per layer')

plt.tight_layout()
plt.show()